# Data Cleaning for Google Data Analytics Capstone Project (Cyclistic bike-share program)


## Discription

This Notebook contains the **Data Cleaning** process for **Google Data Analytics Capstone Project** for a *fictional* bike-share company. The data, however is real and collected from a bike-sharing service based in Chicago (“DIVVY”).

### Quick introduction for the project:
Cyclistic is a bike-share program launched in 2016 that features more than 5,800 bicycles and 600 docking stations (as of 2021 close to 800). Cyclistic sets itself apart by offering different types of bikes such as reclining bikes, hand tricycles, and cargo bikes, making bike-share more inclusive to people with disabilities and riders who can’t use a standard two-wheeled bike. The bikes can be unlocked from one station and returned to any other station in the system anytime.

Until now, Cyclistic’s marketing strategy relied on building general awareness and appealing to broad consumer segments. One approach that helped make it possible was the flexibility of its pricing plans: single-ride passes, full-day passes, and annual memberships. Customers who purchase single-ride or full-day passes are referred to as casual riders. Customers who purchase annual memberships are Cyclistic members.

Cyclistic’s finance analysts have concluded that annual members are much more profitable than casual riders. Although the pricing flexibility helps Cyclistic attract more customers, Moreno(Director of Marketing) believes that maximizing the number of annual members will be key to future growth. Rather than creating a marketing campaign that targets all-new customers, Moreno believes there is a very good chance to convert casual riders into members. She notes that casual riders are already aware of the Cyclistic program and have chosen Cyclistic for their mobility needs.

There is the need to design marketing strategies aimed at converting casual riders into annual members. 
### The main question I will be working on during the Analysis Phase: 
- How do annual members and casual riders use Cyclistic bikes differently?

However, **first I will need to clean the acquired data** to achieve accurate results of analysis.

## Here is a quick overview of what will be done in this notebook:
-	**loading** the data and **merging** individual datasets into one
-   checking for accuracy of datatypes
-   Checking for duplicates
-	handling **null values** 
-	cleaning station names
-	**unifying** inconsistent station ids
-	eliminating unrealistic rows of data
-	eliminating rides done for bike maintenance

### Now, let us proceed to the first stage, which is loading the data and merging it.



## Step 1: Importing and merging monthly datasets

In [1]:
# First, in this cell we import all the libraries that will be needed during the data cleaning precess

import pandas as pd
import numpy as np
import os
import datetime
import re
import random

In [2]:
# Checking how the csv files from our dataset look

df = pd.read_csv('./Data/CSV/202109-divvy-tripdata.csv')
df.sample(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
192319,CEB48050025A36A2,electric_bike,2021-09-02 08:40:23,2021-09-02 08:52:28,Malcolm X College Vaccination Site,631,Franklin St & Jackson Blvd,TA1305000025,41.877613,-87.673832,41.877503,-87.636202,member
394225,B937974DEC40858D,classic_bike,2021-09-04 11:17:01,2021-09-04 11:40:36,Racine Ave & 18th St,13164,Green St & Randolph St,13053,41.858166,-87.656495,41.883668,-87.648670,casual
604745,46E415F175E5A053,classic_bike,2021-09-01 18:45:32,2021-09-01 18:51:23,Theater on the Lake,TA1308000001,Lincoln Park Conservatory,LP-,41.926277,-87.630834,41.923931,-87.635825,member
570673,BDA33F4D32FE3C11,docked_bike,2021-09-05 12:28:16,2021-09-05 12:34:28,Sheridan Rd & Buena Ave,TA1309000027,Sheffield Ave & Waveland Ave,TA1307000126,41.958494,-87.654966,41.949399,-87.654529,casual
175493,519C7EA5FAEE9545,classic_bike,2021-09-26 10:40:59,2021-09-26 10:46:54,Sheridan Rd & Montrose Ave,TA1307000107,Broadway & Sheridan Rd,13323,41.961670,-87.654640,41.952833,-87.649993,member
121816,C55E58EC39FED592,classic_bike,2021-09-07 20:43:42,2021-09-07 20:47:06,Green St & Madison St,TA1307000120,Aberdeen St & Monroe St,13156,41.881892,-87.648789,41.880419,-87.655519,member
53166,F30B4F1EA14E8B5B,electric_bike,2021-09-15 17:41:32,2021-09-15 17:55:04,University Ave & 57th St,KA1503000071,Shore Dr & 55th St,TA1308000009,41.791457,-87.599904,41.795158,-87.580912,member
311805,264B0CB3F9480B53,classic_bike,2021-09-28 16:06:01,2021-09-28 16:37:33,Clark St & Montrose Ave,KA1503000022,Clark St & Elm St,TA1307000039,41.961588,-87.666036,41.902973,-87.631280,casual
726685,881012C25E1C25CC,classic_bike,2021-09-06 00:33:27,2021-09-06 00:45:16,Wells St & Elm St,KA1504000135,Clinton St & Lake St,13021,41.903222,-87.634324,41.885637,-87.641823,member
685519,CC2CB0CAFE3CC025,electric_bike,2021-09-14 12:12:52,2021-09-14 12:17:57,NaN,NaN,NaN,NaN,41.920000,-87.690000,41.920000,-87.690000,member


Quick look at the sample of datasets shows us already that we have some inconcistencies with the station ids.  
We will need to look further into it. We also see some null values. Now we merge all available datasets together to get one continuous dataset.

In [3]:
#Combining data from files for information from 01/2020 to 09/2021 to have a latger dataset to be able to check differences 
#from year to year if we wanted


files = [file for file in os.listdir('./Data/CSV')]

df = pd.DataFrame()

for file in files:
    data = pd.read_csv('./Data/CSV/'+file)
    df = pd.concat([df, data])

# If you want to save the full dataset as new csv file, you can use this command 
# df.to_csv('Bicycle_data_012020_092021.csv', index=False)

df.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,EACB19130B0CDA4A,docked_bike,2020-01-21 20:06:59,2020-01-21 20:14:30,Western Ave & Leland Ave,239,Clark St & Leland Ave,326,41.9665,-87.6884,41.9671,-87.6674,member
1,8FED874C809DC021,docked_bike,2020-01-30 14:22:39,2020-01-30 14:26:22,Clark St & Montrose Ave,234,Southport Ave & Irving Park Rd,318,41.9616,-87.6660,41.9542,-87.6644,member
2,789F3C21E472CA96,docked_bike,2020-01-09 19:29:26,2020-01-09 19:32:17,Broadway & Belmont Ave,296,Wilton Ave & Belmont Ave,117,41.9401,-87.6455,41.9402,-87.6530,member
3,C9A388DAC6ABF313,docked_bike,2020-01-06 16:17:07,2020-01-06 16:25:56,Clark St & Randolph St,51,Fairbanks Ct & Grand Ave,24,41.8846,-87.6319,41.8918,-87.6206,member
4,943BC3CBECCFD662,docked_bike,2020-01-30 08:37:16,2020-01-30 08:42:48,Clinton St & Lake St,66,Wells St & Hubbard St,212,41.8856,-87.6418,41.8899,-87.6343,member
5,6D9C8A6938165C11,docked_bike,2020-01-10 12:33:05,2020-01-10 12:37:54,Wells St & Hubbard St,212,Desplaines St & Randolph St,96,41.8899,-87.6343,41.8846,-87.6446,member
6,31EB9B8F406D4C82,docked_bike,2020-01-10 13:07:35,2020-01-10 13:12:24,Desplaines St & Randolph St,96,Wells St & Hubbard St,212,41.8846,-87.6446,41.8899,-87.6343,member
7,A2B24E3F9C9720E3,docked_bike,2020-01-10 07:24:53,2020-01-10 07:29:50,Desplaines St & Randolph St,96,Wells St & Hubbard St,212,41.8846,-87.6446,41.8899,-87.6343,member
8,5E3F01E1441730B7,docked_bike,2020-01-31 16:37:16,2020-01-31 16:42:11,Wells St & Hubbard St,212,Desplaines St & Randolph St,96,41.8899,-87.6343,41.8846,-87.6446,member
9,19DC57F7E3140131,docked_bike,2020-01-31 09:39:17,2020-01-31 09:42:40,Clark St & Lake St,38,Orleans St & Merchandise Mart Plaza,100,41.8860,-87.6309,41.8882,-87.6364,member


In [4]:
# Sorting the dataset by start_date

df.sort_values(by=['started_at'], inplace=True)
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
99279,1068AB1B8F12FE23,docked_bike,2020-01-01 00:04:44,2020-01-01 00:17:08,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246,41.936300,-87.652700,41.956100,-87.668800,casual
143776,DCF74A0EB3284B3E,docked_bike,2020-01-01 00:10:37,2020-01-01 00:10:54,Daley Center Plaza,81,Daley Center Plaza,81,41.884200,-87.629600,41.884200,-87.629600,member
143777,4DE50A4FC7687A0D,docked_bike,2020-01-01 00:11:14,2020-01-01 00:15:32,Daley Center Plaza,81,Dearborn St & Van Buren St,624,41.884200,-87.629600,41.876300,-87.629200,member
39811,1C78B5F337CBFC93,docked_bike,2020-01-01 00:11:27,2020-01-01 00:13:15,Sheridan Rd & Irving Park Rd,240,Broadway & Sheridan Rd,256,41.954200,-87.654400,41.952800,-87.650000,member
94468,D231CE7990A3AA52,docked_bike,2020-01-01 00:12:34,2020-01-01 00:14:29,Delano Ct & Roosevelt Rd,626,Wabash Ave & Roosevelt Rd,59,41.867500,-87.632200,41.867200,-87.626000,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
623186,D10FFD88830F50EA,electric_bike,2021-09-30 23:59:16,2021-10-01 00:13:16,Ashland Ave & Division St,13061,Peoria St & Jackson Blvd,13158,41.903486,-87.667872,41.878253,-87.649827,casual
9705,64C8FB2D076664DF,electric_bike,2021-09-30 23:59:18,2021-10-01 00:02:33,Austin Blvd & Lake St,16906,NaN,NaN,41.887941,-87.774474,41.890000,-87.770000,casual
73632,8D3EAC038013987F,docked_bike,2021-09-30 23:59:25,2021-10-01 00:11:41,Pine Grove Ave & Irving Park Rd,TA1308000022,Broadway & Barry Ave,13137,41.954383,-87.648043,41.937582,-87.644098,casual
123219,BDB1A0BB7D48CC3D,electric_bike,2021-09-30 23:59:44,2021-10-01 00:49:47,Elizabeth (May) St & Fulton St,13197,Lake Park Ave & 47th St,TA1308000035,41.886574,-87.658446,41.838263,-87.607463,casual


## Step 2: Checking datatypes for accuracy

In [5]:
# Looking at what data types are present in the dataset and if we need to change them
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7898002 entries, 99279 to 681357
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 843.6+ MB


In [6]:
# Setting start and end dates to 'datetime' format for manipulation

df[['started_at', 'ended_at']] = df[['started_at', 'ended_at']].astype('datetime64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7898002 entries, 99279 to 681357
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 843.6+ MB


## Step 3: Checking for duplicate values in the columns

In [7]:
# Checking for duplicated ride_id values in the dataset

df['ride_id'].duplicated().sum()


209

For all columns except the ride_id the duplicated rows are expected.  
Let's take a look what those duplicated rows look like

In [8]:
# We can see that duplicates have incorrect start/end dates. We will need to drop them

df.loc[df['ride_id'].duplicated(keep=False)].sort_values(by=['ride_id', 'started_at'])

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
106076,021A73F8C18B932D,docked_bike,2020-11-25 16:35:39,2020-11-25 16:48:02,Clark St & Winnemac Ave,325,Ravenswood Ave & Berteau Ave,314,41.973347,-87.667855,41.957921,-87.673567,member
23285,021A73F8C18B932D,docked_bike,2020-12-15 12:15:58,2020-11-25 16:48:02,Clark St & Winnemac Ave,TA1309000035,Ravenswood Ave & Berteau Ave,TA1309000018,41.973347,-87.667855,41.957921,-87.673567,member
74015,0334987B57662109,docked_bike,2020-11-25 16:15:04,2020-11-25 16:22:04,Broadway & Berwyn Ave,294,Lakefront Trail & Bryn Mawr Ave,459,41.978353,-87.659753,41.984037,-87.652310,member
37161,0334987B57662109,docked_bike,2020-12-15 11:56:33,2020-11-25 16:22:04,Broadway & Berwyn Ave,13109,Lakefront Trail & Bryn Mawr Ave,KA1504000152,41.978353,-87.659753,41.984037,-87.652310,member
57814,038CAB4A84D9A56B,docked_bike,2020-11-25 16:08:05,2020-11-25 16:24:56,Wabash Ave & 9th St,321,Wabash Ave & 9th St,321,41.870769,-87.625734,41.870769,-87.625734,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49401,FE36876C99DE6E66,docked_bike,2020-12-15 12:17:51,2020-11-25 16:22:24,Clinton St & Roosevelt Rd,WL-008,State St & Randolph St,TA1305000029,41.867117,-87.641087,41.884730,-87.627734,member
180345,FF1B35D974D5B39E,docked_bike,2020-11-25 19:18:20,2020-11-25 19:24:39,Sedgwick St & Huron St,111,Clark St & Elm St,176,41.894666,-87.638437,41.902973,-87.631280,casual
3998,FF1B35D974D5B39E,docked_bike,2020-12-15 12:09:25,2020-11-25 19:24:39,Sedgwick St & Huron St,TA1307000062,Clark St & Elm St,TA1307000039,41.894666,-87.638437,41.902973,-87.631280,casual
49479,FF49C6C596507F0B,docked_bike,2020-11-25 19:14:15,2020-11-25 19:27:10,Clark St & Randolph St,51,Aberdeen St & Jackson Blvd,21,41.884576,-87.631889,41.877726,-87.654787,member


In [9]:
# Let's drop these duplicate rows
df = df.drop_duplicates('ride_id', keep='first').reset_index(drop=True)
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,1068AB1B8F12FE23,docked_bike,2020-01-01 00:04:44,2020-01-01 00:17:08,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246,41.936300,-87.652700,41.956100,-87.668800,casual
1,DCF74A0EB3284B3E,docked_bike,2020-01-01 00:10:37,2020-01-01 00:10:54,Daley Center Plaza,81,Daley Center Plaza,81,41.884200,-87.629600,41.884200,-87.629600,member
2,4DE50A4FC7687A0D,docked_bike,2020-01-01 00:11:14,2020-01-01 00:15:32,Daley Center Plaza,81,Dearborn St & Van Buren St,624,41.884200,-87.629600,41.876300,-87.629200,member
3,1C78B5F337CBFC93,docked_bike,2020-01-01 00:11:27,2020-01-01 00:13:15,Sheridan Rd & Irving Park Rd,240,Broadway & Sheridan Rd,256,41.954200,-87.654400,41.952800,-87.650000,member
4,D231CE7990A3AA52,docked_bike,2020-01-01 00:12:34,2020-01-01 00:14:29,Delano Ct & Roosevelt Rd,626,Wabash Ave & Roosevelt Rd,59,41.867500,-87.632200,41.867200,-87.626000,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7897788,D10FFD88830F50EA,electric_bike,2021-09-30 23:59:16,2021-10-01 00:13:16,Ashland Ave & Division St,13061,Peoria St & Jackson Blvd,13158,41.903486,-87.667872,41.878253,-87.649827,casual
7897789,64C8FB2D076664DF,electric_bike,2021-09-30 23:59:18,2021-10-01 00:02:33,Austin Blvd & Lake St,16906,NaN,NaN,41.887941,-87.774474,41.890000,-87.770000,casual
7897790,8D3EAC038013987F,docked_bike,2021-09-30 23:59:25,2021-10-01 00:11:41,Pine Grove Ave & Irving Park Rd,TA1308000022,Broadway & Barry Ave,13137,41.954383,-87.648043,41.937582,-87.644098,casual
7897791,BDB1A0BB7D48CC3D,electric_bike,2021-09-30 23:59:44,2021-10-01 00:49:47,Elizabeth (May) St & Fulton St,13197,Lake Park Ave & 47th St,TA1308000035,41.886574,-87.658446,41.838263,-87.607463,casual


## Step 4: Handling null values

In [10]:
# Checking for null values in columns
df.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    550902
start_station_id      551525
end_station_name      602531
end_station_id        602992
start_lat                  0
start_lng                  0
end_lat                 8206
end_lng                 8206
member_casual              0
dtype: int64

In [11]:
# Calculating persentage of null values for each column
null_percentage = (df.isna().sum()/df.count())*100
null_percentage

ride_id               0.000000
rideable_type         0.000000
started_at            0.000000
ended_at              0.000000
start_station_name    7.498437
start_station_id      7.507553
end_station_name      8.259210
end_station_id        8.266051
start_lat             0.000000
start_lng             0.000000
end_lat               0.104011
end_lng               0.104011
member_casual         0.000000
dtype: float64

### The majority of null values occurs in station names and ids
Out of quriosity let's check how missing values are distributed within the dataset.
We do that to ensure that we do not have some months full of missing data, since we used multiple datasets to create the DataFrame

In [12]:
# Creating year-month column to check how many values are missing for each month
df['year_month'] = df['started_at'].dt.strftime('%Y-%m')
df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,year_month
7897788,D10FFD88830F50EA,electric_bike,2021-09-30 23:59:16,2021-10-01 00:13:16,Ashland Ave & Division St,13061,Peoria St & Jackson Blvd,13158,41.903486,-87.667872,41.878253,-87.649827,casual,2021-09
7897789,64C8FB2D076664DF,electric_bike,2021-09-30 23:59:18,2021-10-01 00:02:33,Austin Blvd & Lake St,16906,NaN,NaN,41.887941,-87.774474,41.890000,-87.770000,casual,2021-09
7897790,8D3EAC038013987F,docked_bike,2021-09-30 23:59:25,2021-10-01 00:11:41,Pine Grove Ave & Irving Park Rd,TA1308000022,Broadway & Barry Ave,13137,41.954383,-87.648043,41.937582,-87.644098,casual,2021-09
7897791,BDB1A0BB7D48CC3D,electric_bike,2021-09-30 23:59:44,2021-10-01 00:49:47,Elizabeth (May) St & Fulton St,13197,Lake Park Ave & 47th St,TA1308000035,41.886574,-87.658446,41.838263,-87.607463,casual,2021-09
7897792,A2919B99DDEFA3E6,electric_bike,2021-09-30 23:59:48,2021-10-01 00:05:31,Damen Ave & Cortland St,13133,NaN,NaN,41.916051,-87.677389,41.910000,-87.690000,member,2021-09


In [13]:
# Grouping the null values from the station columns by month

null_by_month = df[['start_station_name',
                    'start_station_id',
                    'end_station_name',
                    'end_station_id']].isnull().groupby(df['year_month']).sum().astype(int)
null_by_month

,start_station_name,start_station_id,end_station_name,end_station_id
year_month,,,,
2020-01,0,0,0,0
2020-02,0,0,0,0
2020-03,0,0,1,1
2020-04,0,0,99,99
2020-05,0,0,321,321
2020-06,0,0,468,468
2020-07,149,152,967,969
2020-08,7595,7691,10035,10110
2020-09,19691,19901,23373,23524


In [14]:
# Additionally counting how many rides total were made in each month

rows_by_month = df['ride_id'].groupby(df['year_month']).count()
rows_by_month

year_month
2020-01    143884
2020-02    139585
2020-03    143418
2020-04     84776
2020-05    200274
2020-06    343005
2020-07    551480
2020-08    622361
2020-09    532958
2020-10    388653
2020-11    259716
2020-12    131364
2021-01     96834
2021-02     49622
2021-03    228496
2021-04    337230
2021-05    531633
2021-06    729595
2021-07    822410
2021-08    804352
2021-09    756147
Name: ride_id, dtype: int64

In [15]:
# Dividing null values by total rides to see the percentage of null values for each month

null_percent_by_month = (null_by_month*100).divide(rows_by_month, axis=0)
null_percent_by_month

,start_station_name,start_station_id,end_station_name,end_station_id
year_month,,,,
2020-01,0.000000,0.000000,0.000000,0.000000
2020-02,0.000000,0.000000,0.000000,0.000000
2020-03,0.000000,0.000000,0.000697,0.000697
2020-04,0.000000,0.000000,0.116778,0.116778
2020-05,0.000000,0.000000,0.160280,0.160280
2020-06,0.000000,0.000000,0.136441,0.136441
2020-07,0.027018,0.027562,0.175346,0.175709
2020-08,1.220353,1.235778,1.612408,1.624459
2020-09,3.694663,3.734065,4.385524,4.413856


### After completing this step we can see, that data starts missing around october of 2020 and percentage of missing data stays relatively consistent till september of 2021

It would be an interesting project to try and figure out why the data is missing and work out some way of recovering it.

However for the purposes of my next analysis project I will drop all the null values, since we still have quite a substantial amount of data to use.

In [16]:
# Let's drop all rows with missing values from our dataset
df.dropna(inplace=True)

#Checking that we deleted all null values
df.isna().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
year_month            0
dtype: int64

Now let's proceed to the next step

## Step 5: Checking for inconsistencies in station names and ids

In [17]:
# Creating a separate table to work on start_stations data

start_stations = df[['start_station_name', 'start_station_id', 'start_lat', 'start_lng']]

start_stations

,start_station_name,start_station_id,start_lat,start_lng
0,Sheffield Ave & Wellington Ave,115,41.936300,-87.652700
1,Daley Center Plaza,81,41.884200,-87.629600
2,Daley Center Plaza,81,41.884200,-87.629600
3,Sheridan Rd & Irving Park Rd,240,41.954200,-87.654400
4,Delano Ct & Roosevelt Rd,626,41.867500,-87.632200
...,...,...,...,...
7897783,Clark St & Lincoln Ave,13179,41.915689,-87.634600
7897786,Wabash Ave & Roosevelt Rd,TA1305000002,41.867227,-87.625961
7897788,Ashland Ave & Division St,13061,41.903486,-87.667872
7897790,Pine Grove Ave & Irving Park Rd,TA1308000022,41.954383,-87.648043


In [18]:
# Dropping the duplicate ['start_station_name', 'start_station_id'] to, in theory, get all unique stations

start_stations = start_stations.drop_duplicates(subset= ['start_station_name', 'start_station_id'])\
.sort_values('start_station_name').reset_index(drop=True)

start_stations

,start_station_name,start_station_id,start_lat,start_lng
0,2112 W Peterson Ave,456,41.991200,-87.683600
1,2112 W Peterson Ave,KA1504000155,41.991208,-87.683610
2,351,351,41.930000,-87.780000
3,63rd St Beach,101,41.781000,-87.576100
4,63rd St Beach,15491,41.781016,-87.576119
...,...,...,...,...
1461,Woodlawn Ave & Lake Park Ave,KA1503000065,41.814111,-87.597016
1462,Woodlawn Ave & Lake Park Ave,413,41.814100,-87.597000
1463,Yates Blvd & 75th St,KA1503000024,41.758768,-87.566440
1464,Yates Blvd & 75th St,396,41.758800,-87.566400


We can immediately see that we have inconcistencies regarding station names and their ids.  
Some staion id's have two completely different id's with different datatypes, others have seemingly identical numericas ids,
but one can be written as a string and the other as a numeric value.  
We need to **thoroughly clean** the stations data if we want to use it in the analysis. It is extremely valuable

In [19]:
# Same with the end stations

end_stations = df[['end_station_name', 'end_station_id', 'end_lat', 'end_lng']]
end_stations.head()

,end_station_name,end_station_id,end_lat,end_lng
0,Ashland Ave & Belle Plaine Ave,246,41.9561,-87.6688
1,Daley Center Plaza,81,41.8842,-87.6296
2,Dearborn St & Van Buren St,624,41.8763,-87.6292
3,Broadway & Sheridan Rd,256,41.9528,-87.6500
4,Wabash Ave & Roosevelt Rd,59,41.8672,-87.6260


In [20]:
# We can see that we have inconcistencies regarding station names and their ids
end_stations = end_stations.drop_duplicates(subset= ['end_station_name', 'end_station_id'])\
.sort_values('end_station_name').reset_index(drop=True)

end_stations

# Morover we can see that numerical station ids have different datatype and therefore 
# identically looking ids can be both str or numbers

,end_station_name,end_station_id,end_lat,end_lng
0,2112 W Peterson Ave,456,41.991200,-87.683600
1,2112 W Peterson Ave,KA1504000155,41.991178,-87.683593
2,63rd St Beach,15491,41.781016,-87.576120
3,63rd St Beach,101,41.781000,-87.576100
4,900 W Harrison St,13028,41.874754,-87.649807
...,...,...,...,...
1455,Woodlawn Ave & Lake Park Ave,413,41.814100,-87.597000
1456,Woodlawn Ave & Lake Park Ave,KA1503000065,41.814093,-87.597005
1457,Yates Blvd & 75th St,KA1503000024,41.758768,-87.566440
1458,Yates Blvd & 75th St,396,41.758800,-87.566400


### To demonstrate just how messy the stations actually are, here are some examples I found while exploring the data:

In [21]:
end_stations

,end_station_name,end_station_id,end_lat,end_lng
0,2112 W Peterson Ave,456,41.991200,-87.683600
1,2112 W Peterson Ave,KA1504000155,41.991178,-87.683593
2,63rd St Beach,15491,41.781016,-87.576120
3,63rd St Beach,101,41.781000,-87.576100
4,900 W Harrison St,13028,41.874754,-87.649807
...,...,...,...,...
1455,Woodlawn Ave & Lake Park Ave,413,41.814100,-87.597000
1456,Woodlawn Ave & Lake Park Ave,KA1503000065,41.814093,-87.597005
1457,Yates Blvd & 75th St,KA1503000024,41.758768,-87.566440
1458,Yates Blvd & 75th St,396,41.758800,-87.566400


In [22]:
# Up to 3 different station id for one station name located around the same area. Possibly, the  stations were slightly
# relocated, which caused the creation of new ids

check_station_id = end_stations.loc[end_stations['end_station_name'] == 'Pine Grove Ave & Irving Park Rd']
check_station_id

,end_station_name,end_station_id,end_lat,end_lng
1038,Pine Grove Ave & Irving Park Rd,TA1308000022,41.954383,-87.648043
1039,Pine Grove Ave & Irving Park Rd,254,41.954400,-87.648000


In [23]:
# In the next cells we also see inconsistencies in station names. Stations that start with 'DuSable' also exist without it
check_station_id = end_stations[end_stations['end_station_name'].str.contains('DuSable')]
check_station_id

,end_station_name,end_station_id,end_lat,end_lng
454,DuSable Lake Shore Dr & Belmont Ave,TA1309000049,41.940775,-87.639192
455,DuSable Lake Shore Dr & Diversey Pkwy,TA1309000039,41.932588,-87.636427
456,DuSable Lake Shore Dr & Monroe St,13300,41.880958,-87.616743
457,DuSable Lake Shore Dr & North Blvd,LF-005,41.911722,-87.626804
458,DuSable Lake Shore Dr & Ohio St,TA1306000029,41.892570,-87.614492
459,DuSable Lake Shore Dr & Wellington Ave,TA1307000041,41.936688,-87.636829
460,DuSable Museum,422,41.791600,-87.607900
461,DuSable Museum,KA1503000075,41.791568,-87.607852


In [24]:
check_station_id = start_stations[start_stations['start_station_name'].str.contains('DuSable')]
check_station_id

,start_station_name,start_station_id,start_lat,start_lng
457,DuSable Lake Shore Dr & Belmont Ave,TA1309000049,41.940775,-87.639192
458,DuSable Lake Shore Dr & Diversey Pkwy,TA1309000039,41.932588,-87.636427
459,DuSable Lake Shore Dr & Monroe St,13300,41.880958,-87.616743
460,DuSable Lake Shore Dr & North Blvd,LF-005,41.911722,-87.626804
461,DuSable Lake Shore Dr & Ohio St,TA1306000029,41.892570,-87.614492
462,DuSable Lake Shore Dr & Wellington Ave,TA1307000041,41.936688,-87.636829
463,DuSable Museum,422,41.791600,-87.607900
464,DuSable Museum,KA1503000075,41.791568,-87.607852


In [25]:
check_station_id = start_stations[start_stations['start_station_name'].str.contains('Lake Shore Dr & Belmont Ave')]
check_station_id

,start_station_name,start_station_id,start_lat,start_lng
457,DuSable Lake Shore Dr & Belmont Ave,TA1309000049,41.940775,-87.639192
758,Lake Shore Dr & Belmont Ave,334,41.940800,-87.639200
759,Lake Shore Dr & Belmont Ave,TA1309000049,41.940769,-87.639228


In [26]:
# However 'DuSable' in DuSable Museum is a part of the actual museum name, so we can't delete it in this instance

check_station_id = start_stations[start_stations['start_station_name'].str.contains('Museum')]
check_station_id

,start_station_name,start_station_id,start_lat,start_lng
463,DuSable Museum,422,41.791600,-87.607900
464,DuSable Museum,KA1503000075,41.791568,-87.607852
529,Field Museum,13029,41.865312,-87.617867
530,Field Museum,97,41.865300,-87.617900
975,Museum of Science and Industry,424,41.791700,-87.583900
976,Museum of Science and Industry,KA1503000074,41.791728,-87.583945


### Other inconsistencies include:
- additional substrings such as (Temp), (*)
- station ids for different stations that looc identical  

As seen below

In [27]:
check_station_id = start_stations[start_stations['start_station_name'].str.contains('Temp')]
check_station_id

,start_station_name,start_station_id,start_lat,start_lng
139,Burling St (Halsted) & Diversey Pkwy (Temp),332,41.933100,-87.647800
161,California Ave & Francis Pl (Temp),13259,41.918491,-87.697422
162,California Ave & Francis Pl (Temp),259,41.918500,-87.697400
537,Franklin St & Adams St (Temp),TA1309000008,41.878528,-87.635876
538,Franklin St & Adams St (Temp),286,41.879400,-87.635500
584,Halsted St & 18th St (Temp),13099,41.858371,-87.646388
1060,Pulaski Rd & Eddy St (Temp),KA1504000141,41.946264,-87.731023
1061,Pulaski Rd & Eddy St (Temp),488,41.946300,-87.731000
1378,Wentworth Ave & 24th St (Temp),132,41.850100,-87.632100
1379,Wentworth Ave & 24th St (Temp),TA1308000026,41.850083,-87.632140


In [28]:
# Check for specific values 
check_station_name = start_stations[start_stations['start_station_id'] == '317']
check_station_name

,start_station_name,start_station_id,start_lat,start_lng
852,Long Ave & Belmont Ave,317,41.94,-87.76


In [29]:
# Here's a quick example of inconsistencies i found while looking through the data. 

# ALL THIS SHOWS US HOW MESSY THE STATION NAMES AND IDS ARE

check_station_id = end_stations[end_stations['end_station_name'].str.contains('Wood St & Taylor St')]
check_station_id

,end_station_name,end_station_id,end_lat,end_lng
1447,Wood St & Taylor St,317,41.869200,-87.671000
1448,Wood St & Taylor St (Temp),13285,41.869265,-87.673731
1449,Wood St & Taylor St (Temp),317,41.869265,-87.673731


In [30]:
check_station_id = start_stations[start_stations['start_station_name'].str.contains('Eggleston Ave & 69th St')]
check_station_id

,start_station_name,start_station_id,start_lat,start_lng
479,Eggleston Ave & 69th St,650,41.768898,-87.635876
480,Eggleston Ave & 69th St,650,41.768898,-87.635876
481,Eggleston Ave & 69th St (*),650,41.768900,-87.635900


In [31]:
check_station_id = end_stations[end_stations['end_station_name'].str.contains('Eggleston Ave & 69th St')]
check_station_id

,end_station_name,end_station_id,end_lat,end_lng
476,Eggleston Ave & 69th St,650,41.768898,-87.635876
477,Eggleston Ave & 69th St,650,41.768898,-87.635876
478,Eggleston Ave & 69th St (*),650,41.768900,-87.635900


### After this exploration I came to the conclusion that it would be better to:
 1) Get rid of the inconsistent station names  
 2) Create new ids for all stations
 
**Note**: I'm doing this because I do not have access to the original database that contains the correct station ids.  
Since I work on an isolated dataset, I think I can allow myself to completely reset the station ids for my analysis.


### Let's get started

## Step 6: Fixing inconsistent station names and ids

### 6.1. : Working on stations data separately (cleaning names, creating new ids)

In [32]:
# Changing the name of the station with '351' in it. Also Changing several different satations with DuSable in the name

# Making start station names consistent by deleting substrings such as (Temp), (*), DuSable
                                                
value_replacements = [("DuSable Museum", "Dusable Museum"),    
                ("351", "Mulligan Ave & Wellington Ave"),
                (" (Temp)", ""),
                (" (*)", ""),
                ("DuSable ", "")]

for values in value_replacements:
    start_stations['start_station_name'] = start_stations['start_station_name'].apply(lambda x: x.replace(values[0], values[1]))
    end_stations['end_station_name'] = end_stations['end_station_name'].apply(lambda x: x.replace(values[0], values[1]))



In [33]:
# Droping ids entirely, dropping duplicated station names

start_names  = start_stations[['start_station_name', 'start_lat', 'start_lng']] 

# I include the coordinates here because I want to later use these unique coordinates for each station in my visualisations

start_names = start_names.drop_duplicates('start_station_name').sort_values('start_station_name').reset_index(drop=True)
start_names = start_names.rename(columns={'start_station_name':'station_name', 'start_lat':'lat', 'start_lng':'lng'})
start_names

,station_name,lat,lng
0,2112 W Peterson Ave,41.991200,-87.683600
1,63rd St Beach,41.781000,-87.576100
2,900 W Harrison St,41.874754,-87.649807
3,Aberdeen St & Jackson Blvd,41.877726,-87.654787
4,Aberdeen St & Monroe St,41.880419,-87.655519
...,...,...,...
774,Woodlawn Ave & 55th St,41.795300,-87.596500
775,Woodlawn Ave & 75th St,41.759160,-87.595751
776,Woodlawn Ave & Lake Park Ave,41.814111,-87.597016
777,Yates Blvd & 75th St,41.758768,-87.566440


In [34]:
# Droping ids entirely, dropping duplicated station names

end_names  = end_stations[['end_station_name', 'end_lat', 'end_lng']]

# I include the coordinates here because I want to later use these unique coordinates for each station in my visualisations

end_names = end_names.drop_duplicates('end_station_name').sort_values('end_station_name').reset_index(drop=True)
end_names = end_names.rename(columns={'end_station_name':'station_name', 'end_lat':'lat', 'end_lng':'lng'})
end_names

,station_name,lat,lng
0,2112 W Peterson Ave,41.991200,-87.683600
1,63rd St Beach,41.781016,-87.576120
2,900 W Harrison St,41.874754,-87.649807
3,Aberdeen St & Jackson Blvd,41.877726,-87.654787
4,Aberdeen St & Monroe St,41.880419,-87.655519
...,...,...,...
770,Woodlawn Ave & 55th St,41.795264,-87.596471
771,Woodlawn Ave & 75th St,41.759200,-87.595800
772,Woodlawn Ave & Lake Park Ave,41.814100,-87.597000
773,Yates Blvd & 75th St,41.758768,-87.566440


To explain why I use separate tables for start and end stations:  
In two cells bellow we can see that some stations do not occur in both tables


In [35]:
end_names[~end_names['station_name'].isin(start_names['station_name'])]

,station_name,lat,lng
517,N Damen Ave & W Wabansia St,41.91,-87.68


In [36]:
start_names[~start_names['station_name'].isin(end_names['station_name'])]

,station_name,lat,lng
14,Archer Ave & 43rd St,41.82,-87.70
454,Lyft Driver Center Private Rack,41.90,-87.66
595,Rockwell St & Archer Ave,41.82,-87.69
602,Sacramento Ave & Pershing Rd,41.82,-87.70
746,Western Ave & 62nd St,41.78,-87.68


### To get the complete bike stations dataset we concatenate them together and drop duplicated rows


In [37]:
station_names = pd.concat([start_names, end_names])

station_names = station_names.drop_duplicates('station_name', keep='first') \
.sort_values(by=['station_name']).reset_index(drop=True)

station_names

,station_name,lat,lng
0,2112 W Peterson Ave,41.991200,-87.683600
1,63rd St Beach,41.781000,-87.576100
2,900 W Harrison St,41.874754,-87.649807
3,Aberdeen St & Jackson Blvd,41.877726,-87.654787
4,Aberdeen St & Monroe St,41.880419,-87.655519
...,...,...,...
775,Woodlawn Ave & 55th St,41.795300,-87.596500
776,Woodlawn Ave & 75th St,41.759160,-87.595751
777,Woodlawn Ave & Lake Park Ave,41.814111,-87.597016
778,Yates Blvd & 75th St,41.758768,-87.566440


In [38]:
# There is also os a station for warehouse of the bikes, lets check it

station_names.loc[station_names['station_name'].str.contains('Warehouse')]

,station_name,lat,lng
47,Base - 2132 W Hubbard Warehouse,41.889955,-87.680651


In [39]:
# Now we generate random unique ids for each of our stations

# Once again, I'm doing this because i do not have acces to the stations dataset and i want to be able 
# to have relationships between tables if I will use it in Power BI

station_ids = random.sample(range(1, 999), len(station_names))
station_names.insert(1, 'station_id', station_ids)
station_names

,station_name,station_id,lat,lng
0,2112 W Peterson Ave,569,41.991200,-87.683600
1,63rd St Beach,72,41.781000,-87.576100
2,900 W Harrison St,66,41.874754,-87.649807
3,Aberdeen St & Jackson Blvd,755,41.877726,-87.654787
4,Aberdeen St & Monroe St,334,41.880419,-87.655519
...,...,...,...,...
775,Woodlawn Ave & 55th St,590,41.795300,-87.596500
776,Woodlawn Ave & 75th St,983,41.759160,-87.595751
777,Woodlawn Ave & Lake Park Ave,602,41.814111,-87.597016
778,Yates Blvd & 75th St,910,41.758768,-87.566440


In [40]:
# It is important to make sure that we did not get any duplicated values

station_names.loc[station_names['station_id'].duplicated()]

,station_name,station_id,lat,lng


### 6.2: Working on the whole dataset (cleaning names, merging ids)

In [41]:
# Now let's work with the original dataset. First we need to change the station names like we did previously
# We repeat the same steps we made when we were cleaning station names

# Remember, we created "value replacements" when we were working on stations tables at the start of Step 6.1
# So now we are applying the same logic to the original dataset

# But let's call this dataset "clean_df", not to lose the original file

clean_df = df

for values in value_replacements:
    clean_df['start_station_name'] = clean_df['start_station_name'].apply(lambda x: x.replace(values[0], values[1]))
    clean_df['end_station_name'] = clean_df['end_station_name'].apply(lambda x: x.replace(values[0], values[1]))


In [42]:
# Quickly make sure that operations were succesfull

clean_df[clean_df['start_station_name'].str.contains('Test')]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,year_month


In [43]:
# Now lets merge stations table with the orinal dataset

# For simplicity of merging I created two columns with different column names

stations_merging_start = station_names[['station_name', 'station_id']].rename(columns = {'station_name':'start_station_name',
                                                                                     'station_id':'new_start_id'})

stations_merging_end = station_names[['station_name', 'station_id']].rename(columns = {'station_name':'end_station_name',
                                                                                     'station_id':'new_end_id'})

stations_merging_start

,start_station_name,new_start_id
0,2112 W Peterson Ave,569
1,63rd St Beach,72
2,900 W Harrison St,66
3,Aberdeen St & Jackson Blvd,755
4,Aberdeen St & Monroe St,334
...,...,...
775,Woodlawn Ave & 55th St,590
776,Woodlawn Ave & 75th St,983
777,Woodlawn Ave & Lake Park Ave,602
778,Yates Blvd & 75th St,910


#### Finally we merge the start and end station tables with the original dataset.
We do the inner join, because we do not want to get additional rows for stations that are not present in the start or end columns.
Joining on station name seems to be a bit of a backwards way, but since we made sure that they are unique and consistent - it works.

In [44]:

clean_df = clean_df.merge(stations_merging_start, how = 'inner', on = 'start_station_name')
clean_df = clean_df.merge(stations_merging_end, how = 'inner', on = 'end_station_name')
clean_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,year_month,new_start_id,new_end_id
0,1068AB1B8F12FE23,docked_bike,2020-01-01 00:04:44,2020-01-01 00:17:08,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246,41.936300,-87.652700,41.956100,-87.668800,casual,2020-01,611,941
1,EA43DCCEB46D431E,docked_bike,2020-03-01 22:42:59,2020-03-01 22:54:09,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246,41.936300,-87.652700,41.956100,-87.668800,casual,2020-03,611,941
2,EF2F40FF73446618,docked_bike,2020-04-25 09:21:38,2020-04-25 09:34:46,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246,41.936300,-87.652700,41.956100,-87.668800,member,2020-04,611,941
3,B587BC0C255D275A,docked_bike,2020-05-24 14:02:05,2020-05-24 14:30:51,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246,41.936300,-87.652700,41.956100,-87.668800,member,2020-05,611,941
4,36FD43FF8048DCD2,docked_bike,2020-07-01 21:18:40,2020-07-01 21:32:51,Sheffield Ave & Wellington Ave,115,Ashland Ave & Belle Plaine Ave,246,41.936266,-87.652662,41.956057,-87.668835,member,2020-07,611,941


In [45]:
# Getting the list of columns to use in the next cell
cols = list(clean_df.columns.values)
cols

['ride_id',
 'rideable_type',
 'started_at',
 'ended_at',
 'start_station_name',
 'start_station_id',
 'end_station_name',
 'end_station_id',
 'start_lat',
 'start_lng',
 'end_lat',
 'end_lng',
 'member_casual',
 'year_month',
 'new_start_id',
 'new_end_id']

In [46]:
# for clarity lets change the columns of our df simply using a list

clean_df = clean_df[['ride_id',
             'rideable_type',
             'started_at',
             'ended_at',
             'start_station_name',
             'new_start_id',
             'end_station_name',
             'new_end_id',
             'start_lat',
             'start_lng',
             'end_lat',
             'end_lng',
             'member_casual']]

clean_df.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,new_start_id,end_station_name,new_end_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,1068AB1B8F12FE23,docked_bike,2020-01-01 00:04:44,2020-01-01 00:17:08,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936300,-87.652700,41.956100,-87.668800,casual
1,EA43DCCEB46D431E,docked_bike,2020-03-01 22:42:59,2020-03-01 22:54:09,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936300,-87.652700,41.956100,-87.668800,casual
2,EF2F40FF73446618,docked_bike,2020-04-25 09:21:38,2020-04-25 09:34:46,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936300,-87.652700,41.956100,-87.668800,member
3,B587BC0C255D275A,docked_bike,2020-05-24 14:02:05,2020-05-24 14:30:51,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936300,-87.652700,41.956100,-87.668800,member
4,36FD43FF8048DCD2,docked_bike,2020-07-01 21:18:40,2020-07-01 21:32:51,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936266,-87.652662,41.956057,-87.668835,member
5,F5D9647DF6567069,docked_bike,2020-07-10 18:31:48,2020-07-10 18:52:01,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936266,-87.652662,41.956057,-87.668835,member
6,3D7A4DBC45C37F3D,docked_bike,2020-07-10 18:35:49,2020-07-10 18:52:08,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936266,-87.652662,41.956057,-87.668835,casual
7,7793BCF69208A8BF,docked_bike,2020-07-11 22:26:29,2020-07-11 22:48:12,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936266,-87.652662,41.956057,-87.668835,casual
8,455FD311FB9F87F1,docked_bike,2020-07-13 13:41:26,2020-07-13 14:04:41,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936266,-87.652662,41.956057,-87.668835,casual
9,3293EDFFD406C9DE,docked_bike,2020-07-13 18:19:33,2020-07-13 18:42:15,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.936266,-87.652662,41.956057,-87.668835,member


In [47]:
clean_df = clean_df.rename(columns = {'new_start_id':'start_station_id', 'new_end_id':'end_station_id'})
clean_df = clean_df.sort_values('started_at').reset_index(drop=True)
clean_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,1068AB1B8F12FE23,docked_bike,2020-01-01 00:04:44,2020-01-01 00:17:08,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.9363,-87.6527,41.9561,-87.6688,casual
1,DCF74A0EB3284B3E,docked_bike,2020-01-01 00:10:37,2020-01-01 00:10:54,Daley Center Plaza,200,Daley Center Plaza,200,41.8842,-87.6296,41.8842,-87.6296,member
2,4DE50A4FC7687A0D,docked_bike,2020-01-01 00:11:14,2020-01-01 00:15:32,Daley Center Plaza,200,Dearborn St & Van Buren St,924,41.8842,-87.6296,41.8763,-87.6292,member
3,1C78B5F337CBFC93,docked_bike,2020-01-01 00:11:27,2020-01-01 00:13:15,Sheridan Rd & Irving Park Rd,803,Broadway & Sheridan Rd,844,41.9542,-87.6544,41.9528,-87.6500,member
4,D231CE7990A3AA52,docked_bike,2020-01-01 00:12:34,2020-01-01 00:14:29,Delano Ct & Roosevelt Rd,205,Wabash Ave & Roosevelt Rd,841,41.8675,-87.6322,41.8672,-87.6260,member


In [48]:
# Checking that all the stations ids merged with the respectful station names
clean_df.isna().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

### Now that we did the fundamental unification of the stations, it's time to prepare the data for analysis further. We will need to:
- Check the trips for wrong dates (start date is later than the end time)
- Get rid of useless rides (too short, unrealistically long?)
- Delete the Test rides and rides to and from the warehouse

### Let's proceed

## Step 7: Cleaning rows of data

In [49]:
# First, checking for rides that end before they begin (which is quite impossible) 

clean_df['trip_duration'] = (clean_df['ended_at'] - clean_df['started_at']).astype('timedelta64[m]')
clean_df.loc[clean_df['trip_duration'] < 0]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration
276290,9461DFF13D8BA8AD,docked_bike,2020-02-28 10:09:43,2020-02-28 10:09:42,HQ QR,404,HQ QR,404,41.889900,-87.680300,41.889900,-87.680300,casual,-1.0
326082,EF0E692D7B4BABB3,docked_bike,2020-03-06 09:32:54,2020-03-06 09:32:52,HQ QR,404,HQ QR,404,41.889900,-87.680300,41.889900,-87.680300,casual,-1.0
350861,D353DE3B13901B98,docked_bike,2020-03-09 14:02:50,2020-03-09 14:02:46,HQ QR,404,HQ QR,404,41.889900,-87.680300,41.889900,-87.680300,casual,-1.0
355789,FC3616DAF48D1E33,docked_bike,2020-03-10 13:02:39,2020-03-10 13:02:28,HQ QR,404,HQ QR,404,41.889900,-87.680300,41.889900,-87.680300,casual,-1.0
355842,D5E25F13DC2C9154,docked_bike,2020-03-10 13:10:45,2020-03-10 13:10:33,HQ QR,404,HQ QR,404,41.889900,-87.680300,41.889900,-87.680300,casual,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7041317,3E68DE63824DF361,electric_bike,2021-09-29 17:44:02,2021-09-29 17:40:51,Indiana Ave & Roosevelt Rd,988,Indiana Ave & Roosevelt Rd,988,41.867908,-87.623047,41.867908,-87.623047,member,-4.0
7041525,3AEF4FE696696C8F,electric_bike,2021-09-29 17:49:18,2021-09-29 17:49:15,Clark St & Lunt Ave,901,Clark St & Lunt Ave,901,42.009073,-87.674217,42.009057,-87.674240,member,-1.0
7041967,448610692B74D6B7,classic_bike,2021-09-29 18:00:15,2021-09-29 17:56:57,Kingsbury St & Erie St,320,Kingsbury St & Erie St,320,41.893808,-87.641697,41.893808,-87.641697,member,-4.0
7042344,7623D85B84ADCC27,classic_bike,2021-09-29 18:09:54,2021-09-29 18:07:08,Pine Grove Ave & Irving Park Rd,291,Pine Grove Ave & Irving Park Rd,291,41.954383,-87.648043,41.954383,-87.648043,member,-3.0


In [50]:
# Dropping rows with inposible timestamps

clean_df.drop(clean_df[clean_df.trip_duration < 0].index, inplace=True)
clean_df[clean_df['trip_duration'] < 0]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration


In [51]:
# Checking for extremely long rides. Seeing that there are fiew of them I might not delete them because they will not change the 
# median of the ride duration too much and I cannot explain their existance (they might be valid)

clean_df.loc[clean_df['trip_duration'] > 10000]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration
4302,C5C656C86C2098A1,docked_bike,2020-01-02 12:05:28,2020-01-16 12:26:28,Dearborn St & Erie St,746,Wells St & Huron St,125,41.894000,-87.629300,41.894700,-87.634400,casual,20181.0
14582,9280527D949FEB8F,docked_bike,2020-01-04 02:46:08,2020-01-15 13:17:03,Dearborn St & Erie St,746,Halsted St & Archer Ave,658,41.894000,-87.629300,41.847200,-87.646800,casual,16470.0
15212,C8E1F273E0FE4E3B,docked_bike,2020-01-04 10:38:43,2020-01-29 21:50:40,Broadway & Barry Ave,124,Ravenswood Ave & Lawrence Ave,202,41.937700,-87.644100,41.969100,-87.674200,casual,36671.0
19634,00F1F172648091A8,docked_bike,2020-01-05 15:50:10,2020-02-02 15:14:45,Humboldt Blvd & Armitage Ave,920,Stave St & Armitage Ave,861,41.917500,-87.701800,41.917700,-87.691400,casual,40284.0
23359,3AF89197E4194178,docked_bike,2020-01-06 11:41:11,2020-02-01 16:52:06,Wabash Ave & Cermak Rd,32,Franklin St & Jackson Blvd,635,41.852600,-87.626500,41.877700,-87.635300,casual,37750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6640621,8BAA2D80763C5529,docked_bike,2021-09-10 02:18:47,2021-09-24 20:28:22,Clark St & Wrightwood Ave,257,Clark St & Drummond Pl,507,41.929546,-87.643118,41.931248,-87.644336,casual,21249.0
6676022,4FFC61082B4E0D6B,docked_bike,2021-09-11 14:45:11,2021-09-22 22:05:37,Western Ave & 111th St,365,Base - 2132 W Hubbard Warehouse,902,41.691211,-87.681297,41.889955,-87.680651,casual,16280.0
6730061,B1774CED9F44E25A,docked_bike,2021-09-13 21:19:35,2021-09-21 16:39:51,Clark St & Lake St,933,Cottage Grove Ave & Oakwood Blvd,847,41.886021,-87.630876,41.822985,-87.607100,casual,11240.0
6758977,0B479098D4FE4CD2,docked_bike,2021-09-15 15:01:51,2021-09-26 10:42:06,Latrobe Ave & Chicago Ave,964,Base - 2132 W Hubbard Warehouse,902,41.894745,-87.756895,41.889955,-87.680651,casual,15580.0


In [52]:
# Checking dataset for rides to and from warehouse or bike-checking stations

clean_df.loc[(clean_df['start_station_name'].str.contains('DIVVY|TEST|Warehouse')) \
             | (clean_df['end_station_name'].str.contains('DIVVY|TEST|Warehouse'))]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration
4326,63068F0D638FEED2,docked_bike,2020-01-02 12:10:22,2020-01-03 13:47:38,Dearborn St & Erie St,746,HUBBARD ST BIKE CHECKING (LBS-WH-TEST),243,41.894000,-87.629300,41.890000,-87.680700,casual,1537.0
49407,58F9C0EE600ACE7A,docked_bike,2020-01-10 09:28:31,2020-01-10 09:46:08,Damen Ave & Pierce Ave,259,HUBBARD ST BIKE CHECKING (LBS-WH-TEST),243,41.909400,-87.677700,41.890000,-87.680700,casual,17.0
50678,7240BD03B9513892,docked_bike,2020-01-10 15:01:02,2020-01-11 20:11:27,Leavitt St & Chicago Ave,668,HUBBARD ST BIKE CHECKING (LBS-WH-TEST),243,41.895500,-87.682000,41.890000,-87.680700,casual,1750.0
88907,C4F4D19D20F53509,docked_bike,2020-01-20 15:52:23,2020-02-15 14:32:06,LaSalle St & Jackson Blvd,83,HUBBARD ST BIKE CHECKING (LBS-WH-TEST),243,41.878200,-87.631900,41.890000,-87.680700,casual,37359.0
88908,711D63ACA859298B,docked_bike,2020-01-20 15:52:38,2020-02-15 14:33:20,LaSalle St & Jackson Blvd,83,HUBBARD ST BIKE CHECKING (LBS-WH-TEST),243,41.878200,-87.631900,41.890000,-87.680700,casual,37360.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7031856,A7416227060AEEE5,electric_bike,2021-09-29 08:44:14,2021-09-29 09:02:43,Laflin St & Cullerton St,341,Base - 2132 W Hubbard Warehouse,902,41.854923,-87.663726,41.889989,-87.680363,member,18.0
7035668,A97439268B486062,classic_bike,2021-09-29 13:40:10,2021-09-29 13:47:08,Ashland Ave & Grand Ave,414,Base - 2132 W Hubbard Warehouse,902,41.891072,-87.666611,41.889955,-87.680651,member,6.0
7048550,A19D1AC60C86C8DF,electric_bike,2021-09-30 06:15:54,2021-09-30 06:21:39,Ashland Ave & Lake St,486,Base - 2132 W Hubbard Warehouse,902,41.885903,-87.667176,41.889932,-87.680298,member,5.0
7054177,230B0E691979EA73,electric_bike,2021-09-30 12:56:56,2021-09-30 13:22:00,Honore St & Division St,727,Base - 2132 W Hubbard Warehouse,902,41.903173,-87.673905,41.889926,-87.680248,member,25.0


In [53]:
# Droping rows with rides related to warehousing and testing of the bikes

clean_df.drop(clean_df.loc[clean_df['end_station_name'].str.contains('DIVVY|TEST|Warehouse')].index, inplace=True)
clean_df.drop(clean_df.loc[clean_df['start_station_name'].str.contains('DIVVY|TEST|Warehouse')].index, inplace=True)

In [54]:
# Checking to see if we done it right

clean_df.loc[(clean_df['start_station_name'].str.contains('DIVVY|TEST|Warehouse')) | \
             (clean_df['end_station_name'].str.contains('DIVVY|TEST|Warehouse'))]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration


## Step 8: Deleting added columns, saving the clean DataFrame in a needed format

In [55]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7052690 entries, 0 to 7067227
Data columns (total 14 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    int64         
 6   end_station_name    object        
 7   end_station_id      int64         
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
 13  trip_duration       float64       
dtypes: datetime64[ns](2), float64(5), int64(2), object(5)
memory usage: 807.1+ MB


In [56]:
# Now we are dropping the extra trip duration column to return the dataset to the original shape

clean_df = clean_df.drop(['trip_duration'], axis=1).reset_index(drop=True)
clean_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,1068AB1B8F12FE23,docked_bike,2020-01-01 00:04:44,2020-01-01 00:17:08,Sheffield Ave & Wellington Ave,611,Ashland Ave & Belle Plaine Ave,941,41.9363,-87.6527,41.9561,-87.6688,casual
1,DCF74A0EB3284B3E,docked_bike,2020-01-01 00:10:37,2020-01-01 00:10:54,Daley Center Plaza,200,Daley Center Plaza,200,41.8842,-87.6296,41.8842,-87.6296,member
2,4DE50A4FC7687A0D,docked_bike,2020-01-01 00:11:14,2020-01-01 00:15:32,Daley Center Plaza,200,Dearborn St & Van Buren St,924,41.8842,-87.6296,41.8763,-87.6292,member
3,1C78B5F337CBFC93,docked_bike,2020-01-01 00:11:27,2020-01-01 00:13:15,Sheridan Rd & Irving Park Rd,803,Broadway & Sheridan Rd,844,41.9542,-87.6544,41.9528,-87.6500,member
4,D231CE7990A3AA52,docked_bike,2020-01-01 00:12:34,2020-01-01 00:14:29,Delano Ct & Roosevelt Rd,205,Wabash Ave & Roosevelt Rd,841,41.8675,-87.6322,41.8672,-87.6260,member


In [57]:
# Creating a dataset to use during further analysis

data_12_months = clean_df.loc[clean_df['started_at'] >= '2020-10-01']
data_12_months

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
2703147,D7AF692CFFAF7D2C,docked_bike,2020-10-01 00:00:06,2020-10-01 00:19:08,Lincoln Ave & Diversey Pkwy,236,Clarendon Ave & Junior Ter,752,41.932225,-87.658617,41.961004,-87.649603,casual
2703148,7B6F00FC8CFBD375,docked_bike,2020-10-01 00:00:30,2020-10-01 00:37:56,Leavitt St & Archer Ave,455,Leavitt St & Archer Ave,455,41.828792,-87.680604,41.828792,-87.680604,casual
2703149,783FEDE9D82022A5,electric_bike,2020-10-01 00:00:43,2020-10-01 00:08:32,Broadway & Cornelia Ave,53,Southport Ave & Belmont Ave,392,41.945549,-87.646465,41.939079,-87.663892,member
2703150,97FE368795B20D9E,electric_bike,2020-10-01 00:00:46,2020-10-01 00:08:11,Broadway & Cornelia Ave,53,Southport Ave & Belmont Ave,392,41.945530,-87.646452,41.939108,-87.663849,member
2703151,5403CD9766829B3D,electric_bike,2020-10-01 00:00:48,2020-10-01 00:12:35,Clark St & Wellington Ave,400,Clark St & Winnemac Ave,984,41.936500,-87.647550,41.973255,-87.669709,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7052685,B70FFAEA8E5A34EB,classic_bike,2021-09-30 23:58:37,2021-10-01 00:15:41,Clark St & Lincoln Ave,78,Ritchie Ct & Banks St,742,41.915689,-87.634600,41.906866,-87.626217,member
7052686,4D248C3CF607D3CB,classic_bike,2021-09-30 23:59:07,2021-10-01 00:04:44,Wabash Ave & Roosevelt Rd,841,State St & Harrison St,852,41.867227,-87.625961,41.874053,-87.627716,member
7052687,D10FFD88830F50EA,electric_bike,2021-09-30 23:59:16,2021-10-01 00:13:16,Ashland Ave & Division St,439,Peoria St & Jackson Blvd,347,41.903486,-87.667872,41.878253,-87.649827,casual
7052688,8D3EAC038013987F,docked_bike,2021-09-30 23:59:25,2021-10-01 00:11:41,Pine Grove Ave & Irving Park Rd,291,Broadway & Barry Ave,124,41.954383,-87.648043,41.937582,-87.644098,casual


#### Finaly we can save the clean dataset as a new *csv* file and use it for futher analysis 
```python
clean_df.to_csv('clean_Cyclistic_bike_data_012020_092021.csv', index=False)
data_12_months.to_csv('clean_Cyclistic_bike_data_102020_092021.csv', index=False)
```

#### If we want to use the clean dataset in Power BI it would be a good idea to make the dataset more lightweight by dropping all the columns relating to the stations. In Power BI we can simply ceate relationships between station tables and the main dataset.

```python
only_ids_12_months_data = data_12_months[['ride_id',
                                          'rideable_type',
                                          'started_at',
                                          'ended_at',
                                          'start_station_id',
                                          'end_station_id',
                                          'member_casual']]
only_ids_12_months_data

# Saving the lightweight file

only_ids_12_months_data.to_csv('clean_Cyclistic_bike_data_ids_only_102020_092021.csv', index=False)

# Saving  the stations as start and end tables

station_names.to_csv('Cyclistic_start_stations.csv', index=False)
station_names.to_csv('Cyclistic_end_stations.csv', index=False)
```

### To recap, the following manipulations were performed on the data:
- removed duplicated ride ids (duplicate rides were with incorrect start/end times)
- removed all rows containing null values (null values in station names, ids and coordinates)
- station names names cleaned from typos and inconsistencies
- inconsistent station ids dropped and new ones created
- rides done for warehousing and maintanance removed
- rides with wrong start and end dates removed (some rides ended earlier than they started)